# Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install

In [2]:
!pip install -U datasets bitsandbytes accelerate transformers peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 17.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl 

# 7.4 QLoRA 미세조정

## 모델

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
)
import torch

model = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    torch_dtype=torch.float16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1"
)

gen_cfg = GenerationConfig(
    max_new_tokens=512,
    do_sample=True,
    top_k=50,
    top_p=0.92,
    return_full_text=False,
    eos_token_id=tokenizer.eos_token_id,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

## 데이터셋

In [4]:
from datasets import load_dataset

dataset = load_dataset("klue", "mrc")
dataset['train'][0]

Generating train split:   0%|          | 0/17554 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5841 [00:00<?, ? examples/s]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
 'news_category': '종합',
 'source': 'hankyung',
 'guid': 'klue-mrc-v1_train_12759',
 'is_impossible': False,
 'question_type': 1,
 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'answers': {'answer_start': [478, 478]

In [5]:
doc = dataset["train"]['question'][0]
messages = [{"role": "user", "content": doc}]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
)

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs.to(model.device),
        generation_config=gen_cfg,
    )
    print(tokenizer.decode(outputs[0].cpu().numpy()))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<bos><start_of_turn>user
북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?<end_of_turn>
<start_of_turn>model
북태평양 기단과 오호츠크해 기단은 각각 북반구와 남반구에 위치하며, 국내에 머무르는 기간은 크게 달라집니다. 북태평양 기단은 태평양에서 발견되며, 북반구에 영향을 미치는 해풍과 기상 조건을 형성합니다. 북태평양 기단은 일반적으로 2~3년의 시간 동안 국내에 머무릅니다. 반면 오호츠크해 기단은 북반구 남부에 위치하며, 남반구의 영향을 받는 해풍과 기상 조건을 형성합니다. 오호츠크해 기단은 일반적으로 2~3년의 시간 동안 국내에 머무릅니다. 이 기간 동안 우리는 이러한 기단의 영향을 받은 기후와 환경을 볼 수 있습니다.<eos>


## QLoRA 미세조정 준비

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    device_map="auto",
    quantization_config=bnb_config,
)
model = get_peft_model(model, lora_config)
tokenizer = AutoTokenizer.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def preprocess(example):
    return tokenizer.apply_chat_template(
        [
            {"role": "user", "content": example['question']},
            {"role": "model", "content": example['answers']['text'][0]},
        ],
        return_dict=True,
    )

dataset = dataset.map(
    preprocess,
    num_proc=2,
    remove_columns=dataset["train"].column_names,
)

Map (num_proc=2):   0%|          | 0/17554 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/5841 [00:00<?, ? examples/s]

## QLoRA 학습

In [13]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling


args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Books/outputs/ckpt",
    max_steps=3000,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_8bit",
    warmup_steps=0.03,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=100,
    push_to_hub=False,
    report_to='none',
)

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
    data_collator=collator,
)
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
100,2.400900
200,2.346300
300,2.273300
400,2.259700
500,2.248000
600,2.191200
700,2.198300
800,2.151600
900,2.170700
1000,2.168800


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=3000, training_loss=2.147007781982422, metrics={'train_runtime': 2788.0739, 'train_samples_per_second': 4.304, 'train_steps_per_second': 1.076, 'total_flos': 5429056807464960.0, 'train_loss': 2.147007781982422, 'epoch': 0.6836048763814515})

## 모델 저장

In [14]:
model = model.merge_and_unload()
model.save_pretrained("/content/drive/MyDrive/Books/outputs/ko-gemma-2b-sum-v1")

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [15]:
!ls -alh /content/drive/MyDrive/Books/outputs/ko-gemma-2b-sum-v1

total 2.1G
-rw------- 1 root root 1.2K May 30 17:04 config.json
-rw------- 1 root root  132 May 30 17:04 generation_config.json
-rw------- 1 root root 2.1G May 30 17:04 model.safetensors


## 추론

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel
import torch

model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Books/outputs/ko-gemma-2b-sum-v1",
)
tokenizer = AutoTokenizer.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
)

# 이전에 만들었던 inputs, gen_cfg 사용

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs.to(model.device),
        generation_config=gen_cfg,
    )
    print(tokenizer.decode(outputs[0].cpu().numpy()))

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<bos><start_of_turn>user
북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?<end_of_turn>
<start_of_turn>model
6~8개월간<end_of_turn>
닋을 풀면서 우리나라가 겨울에 따뜻함을 느끼게 하는 것은?<end_of_turn>
최소온도는?<start_of_turn>
태평양 북서풍의 기온은?<end_of_turn>
온도는 몇도 올라가는가?<end_of_turn>
풍속은 몇 km/s에 도달하나요?<end_of_turn>
태풍으로 인한 바람은 약 몇 m/s에 도달하나요?<end_of_turn>
해상 온도는 몇도까지 내려가는가?<end_of_turn>
온도는 약 몇도까지 올라가는가? îna다? yderligere 해당 기단이 해역에 들어와기 시작한 것은 몇 년인가?<end_of_turn>
해양 온도가 상승한 것은 몇년인가?<end_of_turn>
지하 온도가 상승한 것은 몇 년인가?<end_of_turn>
상층 기온이 상승한 것은 몇 년인가?<end_of_turn>
지구 온도 상승이 발생한 것은 몇년인가?<end_of_turn>
변동된 해양기후는 언제까지 지속되나요?<end_of_turn>
변동된 해양기후가 시작된 것은 언제인가?<end_of_turn>
해양 기후가 변화한 것은 언제인가?<end_of_turn>
지구 온도가 상승한 것은 언제인가? 대기오염은 언제까지 지속될 수 있다고? 언제까지 지속될 수 있다고? 대기 오염은 언제까지 지속될 수 있나요? 대기 오염은 언제까지 지속될 수 있나요? 대기오염은 언제까지 지속될 수 있나요? 대기오염은 언제까지 지속될 수 있나요? 대기오염은 언제까지 지속될 수 있나요? 대기오염은 언제까지 지속될 수 있나요? 대기오염은 언제까지 지속될 수 있나요? 대기오염은 언제까지 지속될 수 있나요? 대기오염은 언제까지 지속될 수 있나요? 대기오염은 언제까지 지속될 수 있나요? 대기오염은 언제까지 지속될 수 있나요? 대기오염은 언
